In [132]:
%matplotlib inline
import pandas as pd
import numpy as np
X = pd.read_csv("data/xSave.csv", index_col=0)
y = pd.read_csv("data/ySave.csv", index_col=0) #, usecols=['YEAR_5'])
columns=['CountryCode', 'IndicatorCode', 'Year', 'Value']
data = pd.read_csv("data/indicators.csv", usecols=(columns))
X.head()

,OBSERVATION,NY.GDP.MKTP.CD_0,NY.GDP.PCAP.CD_0,AG.PRD.CREL.MT_0,AG.YLD.CREL.KG_0,AG.PRD.CROP.XD_0,AG.PRD.FOOD.XD_0,AG.LND.CREL.HA_0,AG.PRD.LVSK.XD_0,EN.POP.DNST_0,...,SP.DYN.TFRT.IN_4,TX.VAL.MRCH.CD.WT_4,TX.VAL.MRCH.WL.CD_4,TX.VAL.MRCH.RS.ZS_4,TX.VAL.MRCH.HI.ZS_4,TM.VAL.MRCH.CD.WT_4,TM.VAL.MRCH.WL.CD_4,TM.VAL.MRCH.RS.ZS_4,TM.VAL.MRCH.HI.ZS_4,TG.VAL.TOTL.GD.ZS_4
0,AFG_1971,1.673333e+09,161.384702,3826000.0,1224.516,83.05,70.20,3124500.0,60.38,15.881812,...,7.450,100000000.0,99900000.0,47.314314,42.712713,141000000.0,145386000.0,32.425405,57.253106,13.161423
1,AFG_1977,1.733333e+09,144.173944,4270000.0,1279.592,91.25,73.56,3337000.0,58.45,18.415149,...,7.449,306000000.0,313137000.0,45.115397,35.590492,328000000.0,458652000.0,45.553055,45.343964,21.467268
2,AFG_1978,2.155555e+09,175.027098,4351000.0,1301.915,94.92,76.02,3342000.0,61.00,18.864003,...,7.449,321000000.0,269156000.0,43.718141,27.069432,395000000.0,450516000.0,53.467801,34.542835,21.696969
3,AFG_1979,2.366667e+09,188.085136,4481000.0,1316.392,97.68,78.24,3404000.0,63.38,19.273587,...,7.449,474000000.0,435015000.0,60.016551,18.152248,425000000.0,551216000.0,68.909647,24.561515,24.310830
4,AFG_1980,2.555556e+09,199.164809,4624000.0,1362.404,102.93,83.72,3394000.0,69.07,19.654077,...,7.449,670000000.0,670474000.0,64.592065,19.471896,841000000.0,494754000.0,60.669343,32.162044,41.491344


In [133]:
y.head()

,YEAR_0,YEAR_1,YEAR_2,YEAR_3,YEAR_4
0,145.3210,146.2150,147.1090,148.0030,148.8970
1,150.8782,151.9654,153.0526,154.1398,155.2270
2,151.9654,153.0526,154.1398,155.2270,156.3758
3,153.0526,154.1398,155.2270,156.3758,157.5246
4,154.1398,155.2270,156.3758,157.5246,158.6734


In [134]:
# Can use OBSERVATION to get new y Data if we want to train on something else
predictionFeature = 'SP.ADO.TFRT'
pData = data[data.IndicatorCode == predictionFeature]
pData.head()

observations = [observation.split('_') for observation in X.OBSERVATION.values]
values = [pData[(pData.CountryCode == value[0]) & (pData.Year == int(value[1]))].Value.values[0] for value in observations]
#y = pd.DataFrame(values, index=X.OBSERVATION.values, columns=['Value'])

In [135]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y.YEAR_4, test_size=0.2, random_state=0)

In [136]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import cross_val_score
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

ValueError: could not convert string to float: 'MWI_1988'

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression(normalize=True)
clf.fit(X_train, y_train)
cv_scores = cross_val_score(clf, X_train, y_train, cv=5)
cv_scores

In [ ]:
from sklearn.svm import SVR
clf = SVR()
clf.fit(X_train, y_train)
cv_scores = cross_val_score(clf, X_train, y_train, cv=5)
cv_scores

In [78]:
param_grid = [
    {'C': [10], 'kernel': ['linear']},
    {'C': [10], 'gamma': [0.03], 'kernel': ['rbf']},
]
svm_grid = GridSearchCV(SVR(), param_grid, cv=4)
svm_grid = svm_grid.fit(X_train, y_train)
svm_grid = svm_grid.best_estimator_
svm_grid.score(X_test, y_test)

0.99334943349973914